In [1]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import os
from datetime import datetime as dt
import datetime

In [7]:
#DEFINE FUNÇÕES PRINCIPAIS PARA ACESSO DO BANCO DE DADOS COM A BASE INICIAL

conn = sqlite3.connect('DIST_DB_ANEEL.db')
cursor = conn.cursor()
conn_bi = sqlite3.connect('BD_POWER_BI_FINAL.db')
cursor_bi = conn_bi.cursor()

In [8]:
#PREPARA TABELA COM INFORMAÇÕES UTEIS PARA ANALISE

tableName1 = "DIST_INADIMPLENCIA"
columnsNecessary = f"SigAgente,AnoIndice,NumPeriodoIndice,VlrIndiceEnviado "

query = f"""
        SELECT {columnsNecessary}
        FROM {tableName1}
        WHERE SigAgente = "ENEL CE"

        """
df = pd.read_sql_query(query,conn)

In [9]:
#AGRUPA POR ANO O TOTAL EM INADIMPLENCIAS FINANCEIRAS

df_analise = df.copy()
df_analise['VlrIndiceEnviado'] = df_analise['VlrIndiceEnviado'].astype(str)
df_analise['VlrIndiceEnviado'] = df_analise['VlrIndiceEnviado'].apply(lambda x: float(x.replace(",",".")))
df_analise = df_analise.groupby(['SigAgente','AnoIndice'],)['VlrIndiceEnviado'].sum()
df_analise = df_analise.reset_index()
df_analise['AumentoPercentual'] = df_analise['VlrIndiceEnviado'].pct_change()
df_analise['refDate'] = df_analise['AnoIndice']
df_analise['refDate'] = df_analise['refDate'].apply(lambda x: "01-01-"+str(x))
df_analise['refDate'] = pd.to_datetime(df_analise['refDate'])

print(df_analise)

  SigAgente  AnoIndice  VlrIndiceEnviado  AumentoPercentual    refDate
0   ENEL CE       2014        1125272.88                NaN 2014-01-01
1   ENEL CE       2015        1250471.39           0.111261 2015-01-01
2   ENEL CE       2016        2106050.49           0.684205 2016-01-01
3   ENEL CE       2017        1740182.25          -0.173722 2017-01-01
4   ENEL CE       2018        1430829.94          -0.177770 2018-01-01
5   ENEL CE       2019        1467784.38           0.025827 2019-01-01
6   ENEL CE       2020         681336.72          -0.535806 2020-01-01
7   ENEL CE       2021        2536490.11           2.722814 2021-01-01
8   ENEL CE       2022        3066215.65           0.208842 2022-01-01


In [10]:
#SOBE PARA O BANCO DE DADOS
df_analise.to_sql('Inadimplencia',conn_bi, if_exists='replace', index=False)

conn.close()
conn_bi.close()